In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import time
import random
import csv
import pandas as pd

In [2]:
# 필수 입력
area_select = ['서울', '경기'] # 전국 / 서울, 경기, 인천, 부산, 대구, 광주, 대전, 울산, 세종, 강원, 경남, 경북, 전남, 전북, 충북, 제주
company_name = '삼성'


In [3]:
driver = webdriver.Chrome(executable_path=r'/Users/hyeseon/Documents/chromedriver')
driver.get("https://www.saramin.co.kr/zf_user/")


In [4]:
#검색창 clear
elem = driver.find_element_by_id('ipt_keyword_recruit')
elem.clear()

#검색어 입력
elem.send_keys(company_name)


In [5]:
# 검색클릭
elem = driver.find_element_by_id("btn_search_recruit")
elem.click()

time.sleep(random.randint(1, 3))

In [6]:
cha = ['Y', 'Y', 'N']

In [7]:
# 채용정보 더보기 후 첫페이지로 이동
elem = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[2]/div/a')
elem.click()


In [8]:
elem = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[2]/div/a[1]')
elem.click()

In [9]:
# 처음 100개를 받아오기
elem = driver.find_element_by_xpath("//*[@id='recruit_info']/div[2]/div/div[3]/button")
elem.click()


In [10]:
elem = driver.find_element_by_xpath("//*[contains(text(), '100개씩')]")
elem.click()
time.sleep(random.randint(1, 3))

In [11]:
# 제일 상단으로 이동
driver.execute_script("window.scrollTo(0, 0);")

In [12]:
# 지역 선택으로 받을 것
elem = driver.find_element_by_class_name('area_section')
elem.click()

In [13]:

# 지역 펼쳐보기
elem = driver.find_element_by_class_name('btn_all_category')
elem.click()


In [14]:
area = driver.find_element_by_xpath('//*[@id="sp_main_wrapper"]/div[2]/div/div[2]/div[2]/div[1]/div[2]/div/ul[1]')
area = area.find_elements_by_tag_name("li")


In [15]:
area_list = []
xpath_list = []
for j in range(len(area)):
    area_id = area[j].get_attribute("id")
    
    if area_id == "":
        continue

    xpath = '//*[@id="'+area_id+'"]'
    title = area[j].find_element_by_tag_name('span').text
    area_list.append(title)
    xpath_list.append([xpath, title])
time.sleep(random.randint(1, 3))

In [16]:
area_path_table = pd.DataFrame(xpath_list, columns =['xpath', 'area'])
area_path_table['selected']  = area_path_table['area'].map(lambda x : 1 if '전국' in area_select else 1 if x in area_select else 0)

In [17]:
# 선택된 지역 기준으로 채용 공고를 선택하게 만들기.

In [18]:
area_list = list(area_path_table[area_path_table['selected']==1]['xpath'])

In [19]:
for area in area_list:
    elem = driver.find_element_by_xpath(area)
    elem.click()

In [20]:
# 지역 선택 닫기
elem = driver.find_element_by_class_name('area_section')
elem.click()

In [21]:
time.sleep(random.randint(1, 3))

In [22]:
# 일반 채용정보 / 파견.대행 / 헤드헌팅 중 선택하는 경우를 리스트로 받음
# 전체 채용 정보일 경우 pass로
if cha == ['Y', 'Y', 'Y'] :
    pass

# 최소한 하나 이상의 채용 조건을 선택해야 함
if cha[0] + cha[1] + cha[2] == 'NNN' : 
    print('error')

# 일반 채용정보 선택 해제 
if cha[0] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[1]')
    elem.click()
    
if cha[1] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[2]')
    elem.click()
    
if cha[2] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[3]')
    elem.click()


In [23]:
# 정렬 순서 열기
elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[2]/div/div[2]/button')
elem.click()


In [24]:
# 정렬 순서 1 관련도/2 정확도/3 등록일/4 수정일/5 마감일/6 지원자/7 사원수
orderby = 2
order_xpath = '//*[@id="recruit_info"]/div[2]/div/div[2]/div/ul/li[{}]'.format(str(orderby))

In [25]:
# 정렬 순서 클릭
elem = driver.find_element_by_xpath(order_xpath)
elem.click()

In [26]:
# 공고 개수 확인
recruit = driver.find_element_by_id('recruit_info_list')
recruit = recruit.find_elements_by_tag_name('h2')
content_count = len(recruit)

In [27]:
time.sleep(random.randint(1, 3))

In [28]:
full_dataset = pd.DataFrame(columns = ['title', 'title_link', 'end_date', 'text_info','work_keyword','office_name','office_link'])

In [29]:
recruit = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[1]')

In [30]:
for item_num in range(1,content_count+1):
    # 공고 제목 데이터 선택
    office1 = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[1]/div[{}]'.format(str(item_num)))
    
    # 공고명
    title = office1.find_element_by_tag_name('a').get_attribute("title")
    
    # 공고링크
    title_link = office1.find_element_by_tag_name('a').get_attribute("href")
    
    # 공고 마감일
    end_date = office1.find_element_by_class_name("date").text
    
    # 공고 지역\n경력\n학력\n직업 종류 + 연봉 있는경우
    text_info = office1.find_element_by_class_name("job_condition").text

    # 업종 ' 전까지는 업종 키워드
    work_keyword = office1.find_element_by_class_name("job_sector").text

    
    # 기업명 선택
    office2 = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[1]/div[{}]/div[2]/strong'.format(str(item_num)))
    # 기업명 text
    office_name = office2.find_element_by_tag_name('a').get_attribute("title")
    # 기업 링크
    office_link = office1.find_element_by_tag_name('a').get_attribute("href")
    
    temp_dataset = pd.DataFrame({'title':[title], 'title_link':[title_link], 'end_date':[end_date], 'text_info':[text_info],'work_keyword':[work_keyword],'office_name':[office_name], 'office_link':[office_link]})
    
    full_dataset = pd.concat([full_dataset, temp_dataset], axis = 0)

In [31]:
time.sleep(random.randint(1, 3))
full_dataset = full_dataset.reset_index(drop=True)
full_dataset

,title,title_link,end_date,text_info,work_keyword,office_name,office_link
0,르노삼성자동차 송탄대리점 영업사원 채용 [월 500~700],http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 10/04(일),경기 평택시\n경력무관\n초대졸↑\n계약직·파견직,"자동차매장, 자동차영업 '삼성' 검색위치 근무 부서 , 지원 분야 , 공고 내용 ,...",르노삼성자동차 송탄지점,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
1,"르노삼성자동차 서비스 정규직 (신입, 경력) 채용공고",http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 09/05(토),경기 구리시\n경력무관\n학력무관\n정규직,"경정비, 카센터, A·S센터 외 '삼성' 검색위치 지원 분야 , 공고 내용 , 기업...",르노삼성자동차서비스코너교문점,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
2,삼성가전배송 경력무관 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,오늘마감,대구 달서구\n경력무관\n학력무관\n정규직\n200 만원,"배송·택배, 운송·운수, 물류센터 외 '삼성' 검색위치 지원 분야 , 공고 내용 ,...",삼성물류센타 다존텍(황현찬),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
3,삼성생명GFC사업부 재무전문가의 개인비서 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 09/20(일),서울 강남구\n경력무관\n초대졸↑\n계약직·인턴직,"생명보험, 보험사, 임원비서 외 '삼성' 검색위치 지원 분야 , 공고 내용 , 기업...",삼성생명보험(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
4,"삼성 디지털프라자 서구점 정규직 모집(영업, 경리)",http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 10/12(월),"인천 서구\n신입·경력\n고졸↑\n정규직\n3,000 만원","IT·통신기기판매, 가전·오디오판매, 매장판매 외 '삼성' 검색위치 근무 부서 , ...",삼성서구전자(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
...,...,...,...,...,...,...,...
95,삼성전자 가전제품 배송 설치 보조기사님 모집합니다,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 10/13(화),경남 양산시\n경력무관\n학력무관\n정규직\n180 만원,"배송·택배, 운송·운수, 배송·운송 '삼성' 검색위치 공고 내용 , 담당 업무 , ...",(주)대륙,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
96,[삼성 현장]안전감시단 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 08/31(월),충남 아산시\n신입·경력\n학력무관\n정규직,"용역업체, 보안·경비업체, 시설관리 외 '삼성' 검색위치 공고 내용 , 담당 업무 ...",(주)에스가드,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
97,삼성 반도체 평택 사업장 전자직발 협력사 안전관리원모집,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,내일마감,"경기 평택시\n경력무관\n학력무관\n정규직\n3,400~3,600","파견대행, 안전 '삼성' 검색위치 공고 내용 , 담당 업무 , 공고 제목",(주)제이에스씨앤에스,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
98,[삼성협력사] 자동화설비 엔지니어 모집(3명) - 경력무관,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,~ 10/19(월),충남 아산시\n경력무관\n학력무관\n정규직,"자동제어, 자동화설비 '삼성' 검색위치 공고 내용 , 담당 업무 , 공고 제목",이엠에스(EMS),http://www.saramin.co.kr/zf_user/jobs/relay/vi...


In [32]:
### \n으로 구분되어 있는거 분리하고, 업종 키워도드 새로운 열로 분리

In [33]:
# 날짜 유형 통일
from datetime import datetime,timedelta

In [34]:
end_date_after = []
for row_num in range(len(full_dataset)):
    end_date_before = full_dataset['end_date'][row_num]
    
    # 오늘
    if '오늘' in end_date_before:
        end_date_after.append(datetime.today().strftime("%Y/%m/%d"))
    elif '내일' in end_date_before:
        end_date_after.append((datetime.today() + timedelta(1)).strftime("%Y/%m/%d"))
    elif '상시' in end_date_before:
        end_date_after.append('상시')
    elif '채용시'  in end_date_before:
        end_date_after.append('채용시')

    elif '~' in end_date_before:
        year = datetime.today().year
        month = full_dataset['end_date'][row_num][2:4]
        day = full_dataset['end_date'][row_num][5:7]
        date_after = str(year)+str(month)+str(day)
        end_date_after.append(pd.to_datetime(date_after).strftime("%Y/%m/%d"))
    
    else : end_date_after.append(str(end_date_before))
        

In [35]:
full_dataset['end_date'] = end_date_after

In [36]:
full_dataset

,title,title_link,end_date,text_info,work_keyword,office_name,office_link
0,르노삼성자동차 송탄대리점 영업사원 채용 [월 500~700],http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/10/04,경기 평택시\n경력무관\n초대졸↑\n계약직·파견직,"자동차매장, 자동차영업 '삼성' 검색위치 근무 부서 , 지원 분야 , 공고 내용 ,...",르노삼성자동차 송탄지점,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
1,"르노삼성자동차 서비스 정규직 (신입, 경력) 채용공고",http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/09/05,경기 구리시\n경력무관\n학력무관\n정규직,"경정비, 카센터, A·S센터 외 '삼성' 검색위치 지원 분야 , 공고 내용 , 기업...",르노삼성자동차서비스코너교문점,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
2,삼성가전배송 경력무관 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/08/26,대구 달서구\n경력무관\n학력무관\n정규직\n200 만원,"배송·택배, 운송·운수, 물류센터 외 '삼성' 검색위치 지원 분야 , 공고 내용 ,...",삼성물류센타 다존텍(황현찬),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
3,삼성생명GFC사업부 재무전문가의 개인비서 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/09/20,서울 강남구\n경력무관\n초대졸↑\n계약직·인턴직,"생명보험, 보험사, 임원비서 외 '삼성' 검색위치 지원 분야 , 공고 내용 , 기업...",삼성생명보험(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
4,"삼성 디지털프라자 서구점 정규직 모집(영업, 경리)",http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/10/12,"인천 서구\n신입·경력\n고졸↑\n정규직\n3,000 만원","IT·통신기기판매, 가전·오디오판매, 매장판매 외 '삼성' 검색위치 근무 부서 , ...",삼성서구전자(주),http://www.saramin.co.kr/zf_user/jobs/relay/vi...
...,...,...,...,...,...,...,...
95,삼성전자 가전제품 배송 설치 보조기사님 모집합니다,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/10/13,경남 양산시\n경력무관\n학력무관\n정규직\n180 만원,"배송·택배, 운송·운수, 배송·운송 '삼성' 검색위치 공고 내용 , 담당 업무 , ...",(주)대륙,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
96,[삼성 현장]안전감시단 채용,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/08/31,충남 아산시\n신입·경력\n학력무관\n정규직,"용역업체, 보안·경비업체, 시설관리 외 '삼성' 검색위치 공고 내용 , 담당 업무 ...",(주)에스가드,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
97,삼성 반도체 평택 사업장 전자직발 협력사 안전관리원모집,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/08/27,"경기 평택시\n경력무관\n학력무관\n정규직\n3,400~3,600","파견대행, 안전 '삼성' 검색위치 공고 내용 , 담당 업무 , 공고 제목",(주)제이에스씨앤에스,http://www.saramin.co.kr/zf_user/jobs/relay/vi...
98,[삼성협력사] 자동화설비 엔지니어 모집(3명) - 경력무관,http://www.saramin.co.kr/zf_user/jobs/relay/vi...,2020/10/19,충남 아산시\n경력무관\n학력무관\n정규직,"자동제어, 자동화설비 '삼성' 검색위치 공고 내용 , 담당 업무 , 공고 제목",이엠에스(EMS),http://www.saramin.co.kr/zf_user/jobs/relay/vi...


In [37]:
area =[]
level = []
school = []
keyword = []
for row_num in range(len(full_dataset)):
    area_ = full_dataset['text_info'][row_num].split(sep='\n')[0]
    level_ = full_dataset['text_info'][row_num].split(sep='\n')[1]
    school_ =full_dataset['text_info'][row_num].split(sep='\n')[2]
    keyword_ = full_dataset['work_keyword'][row_num].split(sep="'")[0].strip()
    
    area.append(area_)
    level.append(level_)
    school.append(school_)
    keyword.append(keyword_)

In [38]:
full_dataset['area']= area
full_dataset['level']= level
full_dataset['school']= school
full_dataset['keyword']= keyword

full_dataset = full_dataset.drop(['text_info','work_keyword'],axis=1)


In [39]:
driver.quit()

In [43]:
full_dataset = full_dataset[['title','title_link','area','level','school','end_date','office_name','office_link','keyword']]

In [44]:
full_dataset.to_csv('saramin_crolling.csv',encoding='euckr')